## Dependencies

In [1]:
import glob
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
BERT_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-tf_model.h5'
VOCAB_PATH = '/kaggle/input/bert-base-uncased-huggingface-transformer/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/model_fold_1_last_epoch.h5
/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/model_fold_2_last_epoch.h5
/kaggle/input/146-googleq-a-train-3fold-bert-base-unc-avg/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

## Pre-process data

In [4]:
text_features = ['question_title', 'question_body', 'answer']

# Model parameters

In [5]:
N_CLASS = len(target_cols)
MAX_SEQUENCE_LENGTH = 512

## Test set

In [6]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features
X_test = compute_input_arays(test, text_features, tokenizer, MAX_SEQUENCE_LENGTH)

# Model

In [7]:
bert_config = BertConfig()
bert_config.output_hidden_states=True

def model_fn():
    input_word_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids')
    input_masks = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks')
    segment_ids = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids')

    bert_model = TFBertModel.from_pretrained(BERT_PATH, config=bert_config)
    sequence_output, pooled_output, hidden_states = bert_model([input_word_ids, input_masks, segment_ids])
    
    h12 = tf.reshape(hidden_states[-1][:, 0],(-1, 1, 768))
    h11 = tf.reshape(hidden_states[-2][:, 0],(-1, 1, 768))
    h10 = tf.reshape(hidden_states[-3][:, 0],(-1, 1, 768))
    h09 = tf.reshape(hidden_states[-4][:, 0],(-1, 1, 768))
    
    x = Average()([h12,h11, h10, h09])

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.2)(x)
    output = Dense(N_CLASS, activation="sigmoid", name="output")(x)

    model = Model(inputs=[input_word_ids, input_masks, segment_ids], outputs=output)
    
    return model

# Make predictions

In [8]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += model.predict(X_test) / len(model_path_list)

In [9]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test
submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.945778,0.703718,0.183974,0.444514,0.496109,0.439356,0.663976,0.702761,0.627559,...,0.927892,0.937811,0.578028,0.983120,0.971284,0.857436,0.015603,0.021868,0.912183,0.922433
1,46,0.922644,0.551456,0.006385,0.833000,0.743493,0.902582,0.560596,0.497462,0.034829,...,0.713589,0.939636,0.640355,0.975369,0.983009,0.895015,0.920574,0.093022,0.063952,0.883041
2,70,0.892289,0.593721,0.008883,0.802089,0.910223,0.967434,0.565055,0.461564,0.152277,...,0.863124,0.946007,0.563727,0.974819,0.972842,0.857869,0.071796,0.044341,0.927226,0.922449
3,132,0.901887,0.439599,0.006833,0.795484,0.754222,0.928925,0.545078,0.471406,0.059516,...,0.705886,0.951962,0.670491,0.967558,0.982567,0.903234,0.823360,0.119482,0.596234,0.889489
4,200,0.936757,0.409035,0.025222,0.870807,0.610636,0.835977,0.661967,0.591549,0.115508,...,0.715384,0.879738,0.614173,0.966000,0.960663,0.805240,0.261386,0.106692,0.672906,0.915348


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.902232,0.609651,0.033153,0.769674,0.804574,0.841000,0.574148,0.504893,0.237941,...,0.815973,0.935515,0.651624,0.965831,0.974672,0.868055,0.501625,0.113130,0.512601,0.915946
std,2812.670060,0.048174,0.142209,0.072161,0.118431,0.123519,0.154332,0.057213,0.087597,0.230595,...,0.085293,0.031046,0.049834,0.014918,0.013894,0.049040,0.346985,0.062686,0.307633,0.025688
min,39.000000,0.726270,0.305334,0.002611,0.195304,0.212235,0.140594,0.431704,0.323290,0.005008,...,0.596744,0.747321,0.473963,0.879576,0.896352,0.635330,0.001635,0.005640,0.011530,0.765304
25%,2572.000000,0.874554,0.489754,0.006193,0.705005,0.754069,0.806294,0.533605,0.446378,0.057541,...,0.745366,0.924982,0.618316,0.959962,0.970502,0.846160,0.131911,0.065981,0.218971,0.901539
50%,5093.000000,0.906347,0.603281,0.009509,0.787129,0.809806,0.896633,0.563021,0.478254,0.131573,...,0.820844,0.943313,0.653620,0.968831,0.978557,0.878408,0.547978,0.113267,0.546685,0.920147
75%,7482.000000,0.939305,0.723990,0.021639,0.855133,0.894644,0.941230,0.609216,0.544216,0.367653,...,0.889870,0.955698,0.685886,0.975396,0.983478,0.901144,0.837685,0.152232,0.786044,0.933363
max,9640.000000,0.983029,0.916596,0.709056,0.986514,0.988558,0.988062,0.760667,0.769347,0.934307,...,0.957869,0.984299,0.782480,0.988980,0.993971,0.967699,0.981365,0.317875,0.994661,0.969244
